## MLP w `sklearn`

In [3]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.neural_network import MLPClassifier

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
dataset = np.loadtxt('Dane/pima-indians-diabetes.data', delimiter=",")


X = dataset[:,0:8]
Y = dataset[:,8]

print(X.shape)
print(np.mean(Y))

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)


####
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
####

(768, 8)
0.3489583333333333


In [5]:
model = MLPClassifier(hidden_layer_sizes=(10,))

In [6]:
model.fit(X_train,y_train)

/home/kodolamacz/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [7]:
y_pred = model.predict(X_test)

In [8]:
y_pred.shape

(254,)

In [9]:
print(accuracy_score(y_true=y_test,y_pred=y_pred))

0.7755905511811023


In [10]:
roc_auc_score(y_true=y_test,y_score=model.predict_proba(X_test)[:,1])

0.840848094471283

** Zadanie ** : zbadaj zachowanie sieci w zależności od ilości neuronów

** Uwaga ** zauważ, że nie podaliśmy nigdzie ilości epok. W tej implementacji uczenie odbywa się do momentu, w którym dwie kolejne epoki różnią się od siebie pod kątem funkcji straty o co najwyżej wartość parametru `tol`. Aby zbadać jak różne ilości neuronów zachowują się przy tej samej liczbie iteracji, musimy ustawić `tol=0`

In [17]:
results = []
neurons = [3, 10, 15, 20, 50, 100, 200, 300, 500]

for neuron in neurons:
    model = MLPClassifier(hidden_layer_sizes=(neuron,),tol=0,activation='tanh')
    model.fit(X_train,y_train)
    
    result_dict = {
        'neuron':neuron,
        'accuracy':accuracy_score(y_test,model.predict(X_test)),
        'roc':roc_auc_score(y_test,model.predict_proba(X_test)[:,1])
    }
    
    results.append(result_dict)
    

/home/kodolamacz/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [18]:
df = pd.DataFrame(results)
df.set_index('neuron',inplace=True)
df

,accuracy,roc
neuron,,
3,0.775591,0.830113
10,0.755906,0.833266
15,0.775591,0.838902
20,0.783465,0.837829
50,0.763780,0.833803
100,0.779528,0.839640
200,0.779528,0.834071
300,0.783465,0.830582
500,0.779528,0.828972


Dwa wnioski:
* zbyt długie uczenie prowadzi do przeuczenia
* zbyt dużo neuronów prowadzi do przeuczenia

Zobaczmy, jakby wyglądała sytuacja gdybyśmy zostawili domyślne parametry:


W praktyce stosowanie takiej metody nie jest dobrym pomysłem - przy prostych problemach faktycznie działa, ale przy bardziej skomplikowanych może się okazać że nie dojdziemy do momentu w którym sieć przestanie się uczyć.  

Najcześciej stosowane podejście to ***early stopping***. Polega ono na wydzieleniu części zbioru treningowego jako zbiór walidacyjny, i trenowaniu do momentu w którym przestaniemy obserwować poprawę wyników na zbiorze walidacyjnym.  

Wada takiego rozwiązania? Zmniejszamy zbiór treningowy, więc jeśli nie mamy dużo danych, to potencjalnie tracimy na jakości modelu. Nie warto jednak brać bardzo małego zbioru walidacyjnego, bo wtedy wynik na nim stanie się bardziej losowy i możemy przerwać uczenie w nieoptymalnym momencie.


In [19]:
model_early_stopping = MLPClassifier((5,),activation='tanh',early_stopping=True,validation_fraction=0.2)

In [20]:
model_early_stopping.fit(X_train,y_train)
labels = model_early_stopping.predict(X_test)

In [21]:
accuracy_score(y_test,labels)

0.3031496062992126